# 02. Explore loop census
## Project: Bicycle node network loop analysis

This notebook explores the loop census created in notebook 01, beyond descriptive network statistics.

Contact: Michael Szell (michael.szell@gmail.com)

Created: 2024-01-29  
Last modified: 2024-05-24  

## To do
- [ ] Kommune cohesion: Loops vs distance to boundary
- [X] POI analysis

## Parameters

In [ ]:
loop_numnode_bound = 30
scenario = "scenario2"

restrictions = {
    "scenario1": {  # family with small children
        "looplengthmin": 5000,
        "looplengthmax": 20000,
        "maxslope": 4,
        "waterlengthmax": 10000,
    },
    "scenario2": {  # teenage/adult leisure cyclist
        "looplengthmin": 10000,
        "looplengthmax": 40000,
        "maxslope": 6,
        "waterlengthmax": 10000,
    },
}

PATH = {
    "data_in_network": "../data/input/faxe/network/",
    "data_in_pois": "../data/input/faxe/pois/",
    "data_out": "../data/processed/faxe/",
    "plot": "../plots/faxe/",
}

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import igraph as ig
import pickle
from itertools import compress

## Functions

In [ ]:
%run -i functions.py

## Exploration

### Load data

In [ ]:
with open(
    PATH["data_out"] + "loopcensus_" + str(loop_numnode_bound) + ".pkl", "rb"
) as f:
    allloops = pickle.load(f)
    alllooplengths = pickle.load(f)
    allloopnumnodes = pickle.load(f)
    allloopmaxslopes = pickle.load(f)
    G = pickle.load(f)
    loop_numnode_bound = pickle.load(f)
    nodes_id = pickle.load(f)
    nodes_coords = pickle.load(f)
    numloops = pickle.load(f)
    loopbasis = pickle.load(f)

In [ ]:
# Plot to double-check
plotCheck(G, nodes_id, nodes_coords);

In [ ]:
fig = plt.figure(figsize=(8, 3))
axes1 = fig.add_axes([0.1, 0.1, 0.35, 0.8])
axes2 = fig.add_axes([0.55, 0.1, 0.35, 0.8])

axes1.hist(alllooplengths, density=True)
axes1.set_xlabel("Length [m]")
axes1.set_ylabel("Probability")
axes1.set_title("Loop lengths")

axes2.hist(allloopnumnodes, density=True, bins=list(range(loop_numnode_bound + 1)))
axes2.set_xlabel("Nodes")
axes2.set_title("Nodes per loop")
axes2.set_xlim([0, loop_numnode_bound + 0.5])

plt.text(loop_numnode_bound / 20, 0.01, "Bound: " + str(loop_numnode_bound))
plt.text(loop_numnode_bound / 20, 0.04, "Loops: " + str(numloops));

### Restrict to length bounds

Go through all nodes and restrict their loops to the scenario's length restrictions

In [ ]:
allloops_bound = {}
for sourcenode in allloops:
    this_lengths = np.array(allloops[sourcenode]["lengths"])
    mask_this = (this_lengths >= restrictions[scenario]["looplengthmin"]) & (
        this_lengths <= restrictions[scenario]["looplengthmax"]
    )
    allloops_bound[sourcenode] = {
        "loops": list(compress(allloops[sourcenode]["loops"], mask_this)),
        "lengths": list(compress(allloops[sourcenode]["lengths"], mask_this)),
        "numnodes": list(compress(allloops[sourcenode]["numnodes"], mask_this)),
        "water_profile": list(
            compress(allloops[sourcenode]["water_profile"], mask_this)
        ),
        "poi_diversity": list(
            compress(allloops[sourcenode]["poi_diversity"], mask_this)
        ),
    }

In [ ]:
vertex_sizes = [
    len(allloops_bound[k]["loops"]) for k in range(len(allloops_bound.keys()))
]
numloops_max = max(vertex_sizes)
vertex_sizes = [i / (numloops_max / 20) for i in vertex_sizes]

In [ ]:
fig = plotCheck(G, nodes_id, nodes_coords, vertex_size=vertex_sizes)
plt.text(0, 0.04, "Max # of loops: " + str(numloops_max))
plt.tight_layout()

In [ ]:
fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(restrictions[scenario]["looplengthmin"])
    + "to"
    + str(restrictions[scenario]["looplengthmax"])
)

### Restrict to acceptable gradients

In [ ]:
allloops_bound = {}
for sourcenode in allloops:
    this_lengths = np.array(allloops[sourcenode]["lengths"])
    this_maxslopes = np.array(allloops[sourcenode]["max_slopes"])
    mask_this = this_lengths >= restrictions[scenario]["looplengthmin"]
    mask_this &= this_lengths <= restrictions[scenario]["looplengthmax"]
    mask_this &= this_maxslopes <= restrictions[scenario]["maxslope"]
    allloops_bound[sourcenode] = {
        "loops": list(compress(allloops[sourcenode]["loops"], mask_this)),
        "lengths": list(compress(allloops[sourcenode]["lengths"], mask_this)),
        "numnodes": list(compress(allloops[sourcenode]["numnodes"], mask_this)),
        "water_profile": list(
            compress(allloops[sourcenode]["water_profile"], mask_this)
        ),
        "poi_diversity": list(
            compress(allloops[sourcenode]["poi_diversity"], mask_this)
        ),
    }

In [ ]:
vertex_sizes = [
    len(allloops_bound[k]["loops"]) for k in range(len(allloops_bound.keys()))
]
numloops_max = max(vertex_sizes)
vertex_sizes = [i / (numloops_max / 20) for i in vertex_sizes]

In [ ]:
edge_colors = []
for e in G.es:
    if e["max_slope"] > restrictions[scenario]["maxslope"]:
        edge_colors.append("red")
    else:
        edge_colors.append("grey")

In [ ]:
fig = plotCheck(
    G, nodes_id, nodes_coords, vertex_size=vertex_sizes, edge_color=edge_colors
)
plt.text(
    0,
    0.12,
    "Red: Gradient >" + str(restrictions[scenario]["maxslope"]) + "%",
    fontsize=8,
)
plt.text(0, 0.04, "Max # of loops: " + str(numloops_max))
plt.tight_layout()

In [ ]:
fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(restrictions[scenario]["looplengthmin"])
    + "to"
    + str(restrictions[scenario]["looplengthmax"])
    + "_maxslope"
    + str(restrictions[scenario]["maxslope"])
)

### Restrict to water limits

In [ ]:
allloops_bound_watered = {}
for sourcenode in allloops_bound:
    numloops = len(allloops_bound[sourcenode]["loops"])
    mask_this = [True] * numloops
    for i in range(numloops):
        wp = allloops_bound[sourcenode]["water_profile"][i]
        water_enough = True
        if wp:  # There is water on the way somewhere. Check distances
            for w in wp:
                if w > restrictions[scenario]["waterlengthmax"]:
                    water_enough = False
                    break
            if water_enough and (
                allloops_bound[sourcenode]["lengths"][i] - wp[-1]
                > restrictions[scenario]["waterlengthmax"]
            ):
                water_enough = False
        else:  # No water on the way, so the loop is only valid if short enough
            if (
                allloops_bound[sourcenode]["lengths"][i]
                > restrictions[scenario]["waterlengthmax"]
            ):
                water_enough = False
        mask_this[i] = water_enough
    allloops_bound_watered[sourcenode] = {
        "loops": list(compress(allloops_bound[sourcenode]["loops"], mask_this)),
        "lengths": list(compress(allloops_bound[sourcenode]["lengths"], mask_this)),
        "numnodes": list(compress(allloops_bound[sourcenode]["numnodes"], mask_this)),
        "water_profile": list(
            compress(allloops_bound[sourcenode]["water_profile"], mask_this)
        ),
        "poi_diversity": list(
            compress(allloops_bound[sourcenode]["poi_diversity"], mask_this)
        ),
    }

In [ ]:
vertex_sizes = [
    len(allloops_bound_watered[k]["loops"])
    for k in range(len(allloops_bound_watered.keys()))
]
numloops_max = max(vertex_sizes)
vertex_sizes = [i / (numloops_max / 20) for i in vertex_sizes]

In [ ]:
edge_colors = []
for e in G.es:
    if e["max_slope"] > restrictions[scenario]["maxslope"]:  # and not e["has_water"]:
        edge_colors.append("red")
    # elif e["max_slope"] > restrictions[scenario]["maxslope"] and e["has_water"]:
    #     edge_colors.append("orange")
    elif e["max_slope"] <= restrictions[scenario]["maxslope"] and e["has_water"]:
        edge_colors.append("blue")
    else:
        edge_colors.append("grey")

In [ ]:
fig = plotCheck(
    G, nodes_id, nodes_coords, vertex_size=vertex_sizes, edge_color=edge_colors
)
plt.text(
    0,
    0.17,
    "Blue: Gradient <=" + str(restrictions[scenario]["maxslope"]) + "% and water",
    fontsize=8,
)
# plt.text(0,0.17, "Orange: Gradient >"+ str(restrictions[scenario]["maxslope"])+"% and water", fontsize=8);
plt.text(
    0,
    0.12,
    "Red: Gradient >" + str(restrictions[scenario]["maxslope"]) + "%",
    fontsize=8,
)
plt.text(0, 0.04, "Max # of loops: " + str(numloops_max))
plt.tight_layout()

In [ ]:
fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(restrictions[scenario]["looplengthmin"])
    + "to"
    + str(restrictions[scenario]["looplengthmax"])
    + "_maxslope"
    + str(restrictions[scenario]["maxslope"])
    + "_waterlengthmax"
    + str(restrictions[scenario]["waterlengthmax"])
)

## Restrict with POI diversity

In [ ]:
allloops_bound_watered_diverse = {}
for sourcenode in allloops_bound_watered:
    numloops = len(allloops_bound_watered[sourcenode]["loops"])
    mask_this = [False] * numloops
    for i in range(numloops):
        pd = allloops_bound_watered[sourcenode]["poi_diversity"][i]
        if pd >= 3:
            mask_this[i] = True
    allloops_bound_watered_diverse[sourcenode] = {
        "loops": list(compress(allloops_bound_watered[sourcenode]["loops"], mask_this)),
        "lengths": list(
            compress(allloops_bound_watered[sourcenode]["lengths"], mask_this)
        ),
        "numnodes": list(
            compress(allloops_bound_watered[sourcenode]["numnodes"], mask_this)
        ),
        "water_profile": list(
            compress(allloops_bound_watered[sourcenode]["water_profile"], mask_this)
        ),
        "poi_diversity": list(
            compress(allloops_bound_watered[sourcenode]["poi_diversity"], mask_this)
        ),
    }

In [ ]:
vertex_sizes = [
    len(allloops_bound_watered_diverse[k]["loops"])
    for k in range(len(allloops_bound_watered_diverse.keys()))
]
numloops_max = max(vertex_sizes)
vertex_sizes = [i / (numloops_max / 20) for i in vertex_sizes]

In [ ]:
edge_colors = []
for e in G.es:
    if e["max_slope"] > restrictions[scenario]["maxslope"]:  # and not e["has_water"]:
        edge_colors.append("red")
    # elif e["max_slope"] > restrictions[scenario]["maxslope"] and e["has_water"]:
    #     edge_colors.append("orange")
    elif (
        e["max_slope"] <= restrictions[scenario]["maxslope"]
        and e["has_water"]
        and e["poi_diversity"] >= 3
    ):
        edge_colors.append("blue")
    else:
        edge_colors.append("grey")

In [ ]:
fig = plotCheck(
    G, nodes_id, nodes_coords, vertex_size=vertex_sizes, edge_color=edge_colors
)
plt.text(
    0,
    0.17,
    "Blue: Gradient <="
    + str(restrictions[scenario]["maxslope"])
    + "% and water and diverse",
    fontsize=8,
)
# plt.text(0,0.17, "Orange: Gradient >"+ str(restrictions[scenario]["maxslope"])+"% and water", fontsize=8);
plt.text(
    0,
    0.12,
    "Red: Gradient >" + str(restrictions[scenario]["maxslope"]) + "%",
    fontsize=8,
)
plt.text(0, 0.04, "Max # of loops: " + str(numloops_max))
plt.tight_layout()

In [ ]:
fig.savefig(
    PATH["plot"]
    + "network_numloops_"
    + str(restrictions[scenario]["looplengthmin"])
    + "to"
    + str(restrictions[scenario]["looplengthmax"])
    + "_maxslope"
    + str(restrictions[scenario]["maxslope"])
    + "_waterlengthmax"
    + str(restrictions[scenario]["waterlengthmax"])
    + "_poidiversity3"
)